# Cats and Dogs Problem Solution
The inspiration for this script comes from a beautiful [keras blog](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html).

In [1]:
#Imports 
import os
from random import shuffle

#Keras imports
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [32]:
#First divide train data into 3 parts - train, test, validation. 
#We are doing this for using ImageDataGenarator Class
all_image_names = os.listdir('data/orig/train')
shuffle(all_image_names)

#Send shuffled data into different folders.
l = len(all_image_names)
train = all_image_names[:int(0.80*l)]
validation = all_image_names[int(0.80*l):int(0.90*l)]
test = all_image_names[int(0.90*l):]

os.mkdir('data/new')
os.mkdir('data/new/train')
os.mkdir('data/new/train/cats/')
os.mkdir('data/new/train/dogs')
os.mkdir('data/new/validation')
os.mkdir('data/new/validation/cats')
os.mkdir('data/new/validation/dogs')
os.mkdir('data/new/test')
os.mkdir('data/new/test/cats')
os.mkdir('data/new/test/dogs')

#We must also dividing image into proper subfolders.
#Another prerequisite for ImageDataGenerator Class

for x in train:
    if x.split('.')[0] == 'cat':
        os.rename('data/orig/train/'+x , 'data/new/train/cats/'+x)
    else:
        os.rename('data/orig/train/'+x , 'data/new/train/dogs/'+x)
for x in validation:
    if x.split('.')[0] == 'cat':
        os.rename('data/orig/train/'+x , 'data/new/validation/cats/'+x)
    else:
        os.rename('data/orig/train/'+x , 'data/new/validation/dogs/'+x)
for x in test:
    if x.split('.')[0] == 'cat':
        os.rename('data/orig/train/'+x , 'data/new/test/cats/'+x)
    else:
        os.rename('data/orig/train/'+x , 'data/new/test/dogs/'+x)

In [36]:
#Let us prepare our data
train_conf = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

validation_conf = ImageDataGenerator(rescale=1./255) #Only scaling in test data


train_generator = train_conf.flow_from_directory(
        'data/new/train/',  #Target directory
        target_size=(100, 100),  #All images will be resized to 150x150
        batch_size=40,
        class_mode='binary')  #We will later use binary_crossentropy loss, hence we need binary labels

#Generator for validation data
validation_generator = validation_conf.flow_from_directory(
        'data/new/validation/',
        target_size=(100, 100),
        batch_size=40,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [37]:
#Create model
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(100, 100, 3))) #Convo Layer
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 

model.add(Convolution2D(32, 3, 3)) #Convo Layer
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3)) #Convo Layer
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#The model so far outputs 3D feature maps (height, width, features), we shall flatten those features
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64)) #Fully Connected Layer
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) #Fully Connected Output Layer
model.add(Activation('sigmoid'))

In [38]:
#Compile our model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [39]:
#Let us fit data into the model
model.fit_generator(
        train_generator,
        samples_per_epoch=20000,
        nb_epoch=10,
        validation_data=validation_generator,
        nb_val_samples=800)
model.save_weights('try_1.h5')  # saving weights after training

Epoch 1/10
20000/20000 [==============================] - 97s - loss: 0.6354 - acc: 0.6352 - val_loss: 0.5245 - val_acc: 0.7513
Epoch 2/10
20000/20000 [==============================] - 92s - loss: 0.5430 - acc: 0.7316 - val_loss: 0.5174 - val_acc: 0.7625
Epoch 3/10
20000/20000 [==============================] - 94s - loss: 0.4929 - acc: 0.7676 - val_loss: 0.4247 - val_acc: 0.8150
Epoch 4/10
20000/20000 [==============================] - 90s - loss: 0.4591 - acc: 0.7867 - val_loss: 0.4757 - val_acc: 0.7813
Epoch 5/10
20000/20000 [==============================] - 91s - loss: 0.4394 - acc: 0.8017 - val_loss: 0.3637 - val_acc: 0.8475
Epoch 6/10
20000/20000 [==============================] - 92s - loss: 0.4187 - acc: 0.8136 - val_loss: 0.3895 - val_acc: 0.8275
Epoch 7/10
20000/20000 [==============================] - 92s - loss: 0.4027 - acc: 0.8229 - val_loss: 0.3389 - val_acc: 0.8488
Epoch 8/10
20000/20000 [==============================] - 92s - loss: 0.3914 - acc: 0.8311 - val_loss: 0